# Week 5: Statistical Inference with Python

In week6, we've covered:
* **Variance**:  
    * Measuring Uncertainty  
    * Prior and Posterior Distributions  
    * Credible Intervals  
* **Inference**:  
    * Hypothesis Testing  
    * False Discovery Rate  

The best way to consolidate the knowledge in your mind is by practicing.<br>Please complete the part marked with <span style="color:green">**# TODO**</span>.

[Google](www.google.com) and [Python Documentation](https://docs.python.org/3/contents.html) are your good friends if you have any python questions.

Upload **Week6_Statistical_Inference_With_Python_Homework.ipynb** notebook to your Google Drive and open it with Google Colab

## Inference  

Recall that last week, we were introduced to statistical inference using batting averages from baseball as a running example. We used the binomial distribution to model the likelihood that a player would get a hit during an at bat, given historical data. We used the beta distribution to estimate batting averages and measure our uncertainty.  

This covered the first two articles of this series titled "The beta distribution" and "Emperical Bayes estimation": http://varianceexplained.org/r/simulation-bayes-baseball/  

Let's continue the discussion of variance, and discuss how to use variance to come up with a range for our estimates of batting averages. By the end of this exercise, you should feel comfortable giving a probability that an estimate lies in an interval. Read about credible intervals in the third article of the series: http://varianceexplained.org/r/credible_intervals_baseball/  

### Exercises  

1. Run the codeblock below to reproduce the dataset from last week. Print out `alpha0` and `beta0`.  

In [ ]:
import pandas as pd

# read the dataset
batting_url = 'https://raw.githubusercontent.com/chadwickbureau/baseballdatabank/master/core/Batting.csv'
pitching_url = 'https://raw.githubusercontent.com/chadwickbureau/baseballdatabank/master/core/Pitching.csv'
people_url = 'https://raw.githubusercontent.com/chadwickbureau/baseballdatabank/master/core/People.csv'

batting = pd.read_csv(batting_url)
pitching = pd.read_csv(pitching_url)
master = pd.read_csv(people_url)

# recreate career dataframe
pitchers = pitching['playerID'].tolist()
batting = batting[batting['AB'] > 0]
batting = batting[~batting['playerID'].isin(pitchers)] #filtered out pitchers
batting_sum =batting.groupby(['playerID']).agg({'H':'sum','AB':'sum'}) # get total hits and at bats for each player
batting_sum['Avg'] = batting_sum.loc[:,'H'] / batting_sum.loc[:,'AB'] #calculate the avg batting rate
career = pd.merge(batting_sum, master, how='inner', on='playerID')[["playerID","nameFirst", "nameLast", "H", "AB", "Avg"]]
career= career[career["AB"]>=500]
career.head()

# helper function to estimate priors for alpha and beta
def moments(mu, sigma2):
    alpha = mu**2 * ((1 - mu) / sigma2 - 1 / mu)
    beta = alpha * (1 / mu - 1)
    return (alpha, beta)

# get priors for alpha and beta
empirical_mean = career['Avg'].to_numpy().mean()
empirical_variance = career['Avg'].to_numpy().var()
alpha0, beta0 = moments(empirical_mean, empirical_variance)

# use priors to update estimate of batting average for each player
career['eb'] = ((career.loc[:,'H'] + alpha0) / (career.loc[:,'AB'] + alpha0 + beta0))

# concatenate first and last name
career['name'] = career.loc[:, 'nameFirst'] + ' ' + career.loc[:, 'nameLast']
career.sort_values('eb', ascending=False).loc[:,["playerID", 'name', 'H', 'AB', 'Avg', 'eb']].head(5)

2. Recall `alpha0` and `beta0` represent prior belief for the "average player" as determined by the entire data set. Use your knowledge from last week to generate an array of `1000` samples from a beta distribution with parameters `alpha0` and `beta0`.
    
 2a. Calculate the sample mean and compare it to the ratio $\frac{\alpha_0}{\alpha_0 + \beta_0}$

In [ ]:
# TODO

  2b. Calculate the 0.10 and 0.90 quantlies of the sample.  

In [ ]:
# TODO

3. Find two players in the `career` dataframe that have a similar `Avg` but very different at bats (`AB`).
  
  3a. Create a two-row dataframe that includes only these two players, and display their `name`, `H`, `AB`, `Avg`, and estimated avg (`eb`).

In [ ]:
# TODO

3b. Calculate the *posterior* values for $\alpha$ and $\beta$ for the two players in your subset.

(This Stack Overflow answer details the update process for $\alpha$ and $\beta$: https://stats.stackexchange.com/a/47782)

In [ ]:
# TODO

  3c. Though the `Avg` and the priors (`alpha0`, `beta0`) are similar for your two players, their estimated variance should be very different (due to the difference in at bats). Make an argument (no proof required): Which of your two players will have a lower 0.10 quantile for the posterior distribution of their estimated batting average (`eb`)? Explain your reasoning in a sentence or two.

TODO: Make your argument:

3d. Use your posterior alphas and betas to generate a sample of `1000` estimated batting averages for each of your two players. Calculate the 0.10 and 0.90 quantile for each sample distribution. Compare the outcome to your prediction in 3c. Reconcile any differences.

4. Finally, let's compare confidence and credible intervals.

  4a. For each of your two players, use their `H` and `AB` to calculate the 95% binomial proportion confidence interval.
  
  (The wikipedia entry will be helpful: https://en.wikipedia.org/wiki/Binomial_proportion_confidence_interval. Note that for a 95% confidence interval, the *z* score is 1.96.)

In [ ]:
# TODO

4b. Calculate the 95% credible interval using `alpha` and `beta`. Compare the difference between the confidence interval and the credible interval.

(Here is another resource for understanding confidence vs credible intervals: https://towardsdatascience.com/do-you-know-credible-interval-e5b833adf399)

In [ ]:
# TODO

## Hypothesis Testing

Now that you have a handle on credible intervals, you are ready to use those intervals to test hypotheses.

Read the fourth article to begin to understand how credible intervals can be useful to make decisions: http://varianceexplained.org/r/bayesian_fdr_baseball/

### Exercises  

5. Let's aggregate Hank Aaron's seasons to get his total career at bats.

In [ ]:
h_aaron = career[career['playerID'] == 'aaronha01']
h_aaron.head(5)

5a. Calculate Hank Aaron's total Hits (`H`) and At Bats (`AB`)

In [ ]:
# TODO

5b. Use random sampling to estimate the probability that Hank Aaron's career batting average is below 0.300.

In [ ]:
# TODO

5c. Use random sampling to estimate the probability that the *average* player has a career batting average below 0.300

In [ ]:
# TODO

5d. How would you use the estimates from 4b and 4c to describe how good a hitter Hank Aaron was?

TODO:

6. Find two rows of the `career` dataframe that have the same `eb` estimate (to the third decimal place). Compare the posterior error probability that each of these players has a batting average below 0.300. Reconcile any differences.

## Submission

Download completed **Week_5_Statistical_Inference_With_Python_Homework.ipynb** from Google Colab and commit to your personal Github repo you shared with the faculty.